In [5]:
import cv2
import numpy as np
import os

def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.GaussianBlur(img, (5, 5), 0)
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def process_all_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for img_name in os.listdir(input_folder):
        if img_name.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, img_name)
            processed = preprocess_image(img_path)
            cv2.imwrite(os.path.join(output_folder, img_name), processed)

# Run preprocessing
input_folder = "data/"
output_folder = "processed_data/"
process_all_images(input_folder, output_folder)


In [7]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  

# Train model (CLI alternative)
model.train(data="data.yaml", epochs=50, imgsz=640)


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\ls2823\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 8.87MB/s]


Ultralytics 8.3.86  Python-3.10.16 torch-2.5.1 CPU (13th Gen Intel Core(TM) i7-1365U)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

RuntimeError: Dataset 'data.yaml' error  'data.yaml' does not exist

In [ ]:
results = model("meter_image.jpg", save=True, conf=0.5)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Load dataset (MNIST for now, replace with real meter digits)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Train model
model.fit(train_images[..., np.newaxis], train_labels, epochs=10, validation_data=(test_images[..., np.newaxis], test_labels))

# Save model
model.save("digit_recognition.h5")


In [ ]:
from tensorflow.keras.models import load_model

def predict_digit(img):
    """Predicts a single digit from an image."""
    img = cv2.resize(img, (28, 28))
    img = img / 255.0
    img = img.reshape(1, 28, 28, 1)
    prediction = model.predict(img)
    return np.argmax(prediction)

# Example prediction
digit = predict_digit(processed_img)
print("Predicted Digit:", digit)


In [ ]:
import easyocr

reader = easyocr.Reader(["en"])

def ocr_read_meter(image_path):
    """Extracts text using OCR."""
    result = reader.readtext(image_path, detail=0)
    return [text for text in result if text.replace('.', '').isdigit()]

meter_reading = ocr_read_meter("meter_image.jpg")
print("Detected Meter Reading:", meter_reading)


In [ ]:
def detect_meter_reading(image_path):
    """End-to-end pipeline to read meter values."""
    # Detect meter display with YOLO
    model = YOLO("best.pt")
    results = model(image_path)
    
    for r in results:
        for box in r.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            meter_display = cv2.imread(image_path)[y1:y2, x1:x2]
    
    # OCR Backup
    ocr_results = ocr_read_meter(image_path)
    print("OCR Result:", ocr_results)
    
    # CNN Prediction (Digit by Digit)
    digits = [predict_digit(meter_display)]
    print("Final Meter Reading:", "".join(map(str, digits)))

detect_meter_reading("meter_image.jpg")
